In [13]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [20]:
df = pd.read_csv('proc_dataset.csv')
df.head(3)

,age,weight,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diagnoses,A1Cresult,metformin,glimepiride,glipizide,glyburide,rosiglitazone,insulin,change,diabetesMed,readmitted
0,0-10,NaN,1,41,0,1,1,None,No,No,No,No,No,No,0,0,NO
1,10-20,NaN,3,59,0,18,9,None,No,No,No,No,No,Up,1,1,>30
2,20-30,NaN,2,11,5,13,6,None,No,No,Steady,No,No,No,0,1,NO


# 3.1 ОТБОР ПРИЗНАКОВ

Все необходимые признаки, которые имеют наибольшее влияние на переменную **`readmitted`** были выделены в прошлой сессии.
Данные признаки имеют нормальное распределение значений, а так же имеют достаточно данных для дальнейшей кластеризации.

# 3.2 ВЫБОР И ПОСТРОЕНИЕ НОВЫХ ПЕРЕМЕННЫХ; ТЕСТИРОВАНИЕ ЗНАЧИМОСТИ ВЫБРАННЫХ ПЕРЕМЕННЫХ 

Данные следует обработать, а именно:
- заполнить пустые значения
- привести к единому формату
- нормализовать значения

In [25]:
# заполняю пустые значения
df['A1Cresult'] = df['A1Cresult'].fillna(0)
df['weight'] = df['weight'].fillna(0)

In [22]:
# сохраню преобразованные данные
df.to_csv('preproc_df.csv', index=None)

In [16]:
# привожу к единому формату
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()

str_df = df.select_dtypes(object)
df = df.drop(str_df,1)
X = pd.DataFrame(oe.fit_transform(str_df.astype(str)), columns=str_df.columns) 
df = pd.concat([X, df],1)
df.head()

,age,weight,A1Cresult,metformin,glimepiride,glipizide,glyburide,rosiglitazone,insulin,readmitted,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diagnoses,change,diabetesMed
0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1,41,0,1,1,0,0
1,1.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,3,59,0,18,9,1,1
2,2.0,0.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2,11,5,13,6,0,1
3,3.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,2,44,1,16,7,1,1
4,4.0,0.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1,51,0,8,5,1,1


Для перед кластеризацией нужно нормализовать данные

In [17]:
from sklearn.preprocessing import OneHotEncoder
X = OneHotEncoder().fit_transform(df.drop('readmitted',1))
Y = df['readmitted']

Данные успешно обработаны и приведены к единому формату

# 3.3 АРГУМЕНТИРОВАННЫЙ ВЫБОР АЛГОРИТМА КЛАСТЕРИЗАЦИИ

Наиболее подходящими моделями и методами кластеризации я считаю:
- KMeans - хорошо работает с большим набором данных
- MeanShift - хорошо работает с неравномерными кластерами
- SpectralClustering - хорошо работает с маленьким количеством кластеров

In [7]:
# импорт необходимых библиотек
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
from tqdm import tqdm

In [8]:
# настройка гиперпараметров
km = KMeans(n_clusters=3, random_state=10)
ms = MeanShift(n_jobs=-1)
sc = SpectralClustering(n_clusters=3, n_jobs=-1)

In [9]:
# обучение моделей
iterr = 128
for i in tqdm(range(0,X.shape[0], iterr)):
    km.fit(X[i:i+iterr])
    ms.fit(X[i:i+iterr].toarray())
#     sc.fit(X[i:i+iterr])

100%|████████████████████████████████████████████████████████████████████████████████| 796/796 [03:21<00:00,  3.94it/s]


In [10]:
%time print("Точность KMeans:", silhouette_score(X, km.predict(X)))
%time print("Точность MeanShift:", silhouette_score(X, ms.predict(X)))
%time print("Точность SpectralClustering:", silhouette_score(X[:1000], sc.fit_predict(X[:1000])))

Точность KMeans: 0.06650156583838544
Wall time: 7min 51s
Точность MeanShift: 0.04939702725830733
Wall time: 8min 39s
Точность SpectralClustering: 0.0718759673777761
Wall time: 380 ms


Как можно увидеть спектральная кластеризация показала наилучший результат, по сравнению с другими моделями